In [1]:
import pandas as pd

In [57]:
df=pd.read_csv("datasets\spotify_millsongdata.csv")
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [58]:
df.tail()

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57650 entries, 0 to 57649
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   artist  57650 non-null  object
 1   song    57650 non-null  object
 2   link    57650 non-null  object
 3   text    57650 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB


In [60]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [61]:
df=df.sample(5000).drop(columns=["link"]).reset_index(drop=True)

In [62]:
print(df['text'][0])

When we kiss my heart's on fire  
Burning with a strange desire  
And I know, each time I kiss you  
That your heart's on fire too  
So, my darling, please surrender  
All your love so warm and tender  
Let me hold you in my arms, dear  
While the moon shines bright above  
All the stars will tell a story  
Of a love and all it's glory  
Let us take this night of magic  
And make it a night of love  
  
Won't you please surrender to me  
Your lips, your arms, your heart, dear  
Be mine forever  
Be mine tonight




# Text cleaning/Text preprocessing

In [63]:
df['text']=df['text'].str.lower().replace(r'^a-zA-Z0-9',' ').replace(r'\n',' ',regex=True)#\w\s-->shortcut

In [64]:
df.tail()

,artist,song,text
4995,Randy Travis,Nothing But The Blood,what can wash away my sins? \r nothing but th...
4996,Gino Vannelli,Powerful People,look at the powerful people \r stealing the s...
4997,Lana Del Rey,Beautiful Player,"[uh, what? star, star \r lana del rey, lana d..."
4998,Rage Against The Machine,Clear The Lane,hey yo! \r let me be front \r this is real a...
4999,Olly Murs,Heart Skips A Beat,my heart skips-skips-skips-skips-skips-skips a...


In [65]:
import nltk
from nltk.stem.porter import PorterStemmer

In [66]:
stem=PorterStemmer()

In [67]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sivak\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [68]:
def token(text):
    token=nltk.word_tokenize(text)
    a=[stem.stem(w) for w in token]
    return " ".join(a)

In [69]:
df['text']=df['text'].apply(lambda x: token(x))

In [70]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [71]:
from sklearn.metrics.pairwise import cosine_similarity

In [72]:
tfidf=TfidfVectorizer(analyzer='word',stop_words='english')

In [73]:
matrix=tfidf.fit_transform(df['text'])

In [74]:
matrix

<5000x18286 sparse matrix of type '<class 'numpy.float64'>'
	with 275792 stored elements in Compressed Sparse Row format>

In [75]:
similar=cosine_similarity(matrix)

In [76]:
similar[0]

array([1.        , 0.02818455, 0.        , ..., 0.05129478, 0.01400678,
       0.06413769])

In [78]:
df[df['song']=="Nothing But The Blood"].index[0]

4995

### Recommender function

In [81]:
def recommender(song_name):
    idx=df[df['song']==song_name].index[0]
    distance=sorted(list(enumerate(similar[idx])),reverse=True,key=lambda x:x[1])
    song=[]
    for s_id in distance[1:10]:
        song.append(df.iloc[s_id[0]].song)
    return song

In [82]:
recommender("Nothing But The Blood")

['I Believe In Jesus',
 'Jesus, Jesus',
 'You Are',
 'Glorify Your Name',
 'Jesus The Same',
 'Jesus Is A Soul Man',
 "He's Enough",
 "It's All In Him",
 'Reason I Live']

In [83]:
import pickle

In [87]:
pickle.dump(similar,open("similarity.pkl","wb"))

In [88]:
pickle.dump(df,open("df.pkl","wb"))